In [ ]:
# Lets start by detecting if we can detect text in the image using tessaract 

In [2]:
import pytesseract
import PIL
from PIL import Image


In [6]:
# testing image a1 for OCR 
img = Image.open('readonly/small_img/a-0.png')
img.show()

In [ ]:
text = pytesseract.image_to_string(img) 

In [ ]:
# lets tes